# 마르코프 국면전환 모델을 이용한 코스피 시장 국면 관측

## 본 코드는 다음의 사이트를 참조하여 제작함을 미리 알려드립니다. 모델의 세부내용은 다음의 사이트에서 확인해주시면 감사하겠습니다.

### 출처 : https://sarit-maitra.medium.com/markov-regime-switching-non-linear-model-8ebfdf6eb755

# 1. 필요 라이브러리 설치 및 데이터 전처리(KOSPI)

## 전처리 과정은 시계열데이터를 노이즈를 감소시키게 하기 위해서 이동평균을 해주었습니다.
## 정상성 역시 만족시키기 위해서 차분을 해줍니다. (pct_change 함수)


In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go

import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_datareader.data as web
import xlrd
# import FinanceDataReader as fdr
# from arch import arch_model

import statsmodels.api as sm
import scipy
%matplotlib inline
plt.style.use('seaborn')

In [13]:
fn= "KOSPI_1995.xlsx"
df_Kospi= pd.read_excel(fn)
df_Kospi= df_Kospi.iloc[:,4:].T

header= df_Kospi.index.tolist()
H= list(map(lambda x: datetime.datetime.strptime(x, "%Y/%m/%d"), header))
df_Kospi.index= H
df_Kospi.columns=["KOSPI"]
df_Kospi.dropna(inplace=True)

df_Kospi= df_Kospi.loc["2001":"2021",:]

ValueError: time data '배당수익률(%)' does not match format '%Y/%m/%d'

In [ ]:
df_Kospi["simple_rtn"]= df_Kospi["KOSPI"].pct_change()
input_data=df_Kospi["rolling_price"].pct_change().dropna()

## 2. 마르코프 국면 전환 모델 사용

### 마르코프 국면은 총 3개로 불황국면 1개, 보통국면1개 ,호황국면 1개로 나누어서 분류되게끔 해주었습니다.

In [ ]:
mod_kospi = sm.tsa.MarkovRegression(input_data, k_regimes=4, trend="n", switching_variance=True)
res_kospi = mod_kospi.fit()

res_kospi.summary()

In [ ]:
fig, axes = plt.subplots(4, figsize=(12, 6))

ax = axes[0]
ax.plot(res_kospi.smoothed_marginal_probabilities[0])
ax.set(title="Smoothed probability of a low-variance regime for stock returns")

ax = axes[1]
ax.plot(res_kospi.smoothed_marginal_probabilities[1])
ax.set(title="Smoothed probability of a high-variance regime for stock returns")

ax = axes[2]
ax.plot(res_kospi.smoothed_marginal_probabilities[2])
ax.set(title="Smoothed probability of a high-variance regime for stock returns")

ax = axes[3]
ax.plot(res_kospi.smoothed_marginal_probabilities[3])
ax.set(title="Smoothed probability of a high-variance regime for stock returns")


fig.tight_layout()

In [ ]:
df_kospi_prob= res_kospi.smoothed_marginal_probabilities
df_kospi_prob.columns=["Low_var1","Low_var2","High_var1","High_var2"]
df_kospi_prob= df_kospi_prob.loc["2018":,:]

In [ ]:
df_kospi_prob.head()

In [14]:
def check_regime2(x):
    if x =="Low_var1":
        return 0
    elif x=="Low_var2":
        return 0.4
    elif x=="High_var1":
        return 0.7
    else:
        return 1

In [ ]:
df_test= df_kospi_prob.copy()

df_test["proba"]= df_test.apply(lambda x: x.max(), axis=1)
df_test["multinom"]= df_test.idxmax(axis=1)

In [ ]:
df_test["regime"]= df_test["multinom"].apply(check_regime2)
df_test.head()

In [ ]:
df_test.groupby("regime")["regime"].count()

In [ ]:
df_test_3= df_test.resample("W-Mon").last()

In [ ]:
df_test_3.head()

In [ ]:
df_test_3.to_csv("4regimes_rolling.csv")